### Character level encoding using Inbuild function

In [1]:
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer

# Define a list of text data
text_data = ["This is a sample sentence", "Another example sentence", "One more text sentence", 'this']

In [2]:
enc=CountVectorizer(analyzer='char')

In [4]:
enc_data=enc.fit_transform(text_data)
df=DataFrame(enc_data.toarray(),columns=enc.vocabulary_)
df

,t,h,i,s,,a,m,p,l,e,n,c,o,r,x
0,4,2,1,4,1,2,1,1,2,0,1,0,4,2,0
1,2,2,1,6,1,0,1,1,3,1,1,1,1,2,1
2,3,0,1,6,0,0,0,1,3,2,0,1,1,3,1
3,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0


### Character level encoding from scratch

#### Encoding

In [5]:
# Extract unique characters from the text data
characters = list(set(''.join(text_data)))

# Print the number of unique characters
print(len(characters))

# Create a dictionary mapping characters to integers
chr_to_int = {char: indx for indx, char in enumerate(characters)}

# Convert text data to numerical representation using the dictionary
encoded_data = [[chr_to_int[char] for char in sent] for sent in text_data]

# Print the numerical representation of the text data
print(encoded_data)


18
[[14, 16, 0, 13, 1, 0, 13, 1, 17, 1, 13, 17, 5, 7, 8, 15, 1, 13, 15, 2, 4, 15, 2, 10, 15], [6, 2, 11, 4, 16, 15, 3, 1, 15, 9, 17, 5, 7, 8, 15, 1, 13, 15, 2, 4, 15, 2, 10, 15], [12, 2, 15, 1, 5, 11, 3, 15, 1, 4, 15, 9, 4, 1, 13, 15, 2, 4, 15, 2, 10, 15], [4, 16, 0, 13]]


#### Decoding

In [7]:
# Create a dictionary mapping integers back to characters
int_to_chr = {indx: char for indx, char in enumerate(characters)}

# Revert the numerical representation back to text using the dictionary
decoded_data = [''.join([int_to_chr[char] for char in sent]) for sent in encoded_data]

# Print the decoded text data
print(decoded_data)


['This is a sample sentence', 'Another example sentence', 'One more text sentence', 'this']


### TF-IDF from Scratch

TF-IDF stands for Term Frequency-Inverse Document Frequency, and it is a numerical statistic used in natural language processing and information retrieval to evaluate the importance of a word in a document relative to a collection of documents (corpus).

Here's a breakdown of the components:

1. Term Frequency (TF):
This measures how often a term (word) appears in a document. It is calculated as the number of times a term occurs in a document divided by the total number of words in that document. The idea is that a higher frequency indicates a higher importance.



2. Inverse Document Frequency (IDF):
This measures how important a term is across a collection of documents (corpus). The idea is to give more weight to terms that are rare across the entire corpus but are frequent in specific documents. It is calculated as the logarithm of the total number of documents divided by the number of documents containing the term, plus one to avoid division by zero.



3. TF-IDF Score:


The TF-IDF score reflects how important a word is to a document in the context of a larger collection of documents. Words that are frequent in a document but rare in the corpus will have higher TF-IDF scores.



In [8]:
import numpy as np
import pandas as pd

# Input text data
text_data = ["good movie", "bad movie", "good bad movie"]

# Sentences and Word Tokenization
word_data = [sentence.split(' ') for sentence in text_data]

# Finding the Vocabulary
vocab = list(set([word for sentence in word_data for word in sentence]))

# Finding Term Frequency
tf_data = []
for sentence in word_data:
    tf_sentence = []
    for word in vocab:
        # Calculate Term Frequency (TF) for each word in the sentence
        tf_sentence.append(sentence.count(word) / len(sentence))
    tf_data.append(tf_sentence)

# Finding Inverse Document Frequency
n_documents = len(text_data)
idf_data = []

for word in vocab:
    n_appearances = 0
    for sentence in word_data:
        # Count the number of documents where the word appears
        if word in sentence:
            n_appearances += 1

    # Calculate Inverse Document Frequency (IDF) for each word
    idf = np.log(n_documents / n_appearances)
    idf_data.append(idf)

# Finding TF-IDF for each sentence
tfidf_data = []
for tf_sentence in tf_data:
    tfidf_sentence = []
    for tf, idf in zip(tf_sentence, idf_data):
        # Calculate TF-IDF for each word in the sentence
        tfidf_sentence.append(tf * idf)
    tfidf_data.append(tfidf_sentence)

# Create a DataFrame to display TF-IDF values for each word in each sentence
df = pd.DataFrame(tfidf_data, columns=vocab)
df['full_sent'] = text_data

# Display the resulting DataFrame
print(df)

       good       bad  movie       full_sent
0  0.202733  0.000000    0.0      good movie
1  0.000000  0.202733    0.0       bad movie
2  0.135155  0.135155    0.0  good bad movie


### TF-IDF using Sklearn

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
tfidf=TfidfVectorizer()

In [11]:
tfidf_data=tfidf.fit_transform(text_data)

In [15]:
df=pd.DataFrame(tfidf_data.toarray(),columns=tfidf.vocabulary_)

In [16]:
df

,good,movie,bad
0,0.000000,0.789807,0.613356
1,0.789807,0.000000,0.613356
2,0.619805,0.619805,0.481334


### TF-IDF in Action

In [55]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stemmer=PorterStemmer()

In [56]:
df=pd.read_csv('twitter_validation.csv',header=None)

In [57]:
df.head()

,0,1,2,3
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [58]:
df=df.rename({0:'id',1:'company',2:'sentiment',3:'raw_tweets'},axis=1)

In [59]:
# Filtering out tweets labeled as 'Irrelevant' or 'Neutral' for sentiment analysis
df = df[df['sentiment'] != 'Irrelevant']
df = df[df['sentiment'] != 'Neutral']

In [60]:
tweets=df['raw_tweets'].values

In [61]:
tweets

array(['@Microsoft Why do I pay for WORD when it functions so poorly on my @SamsungUS Chromebook? 🙄',
       "CSGO matchmaking is so full of closet hacking, it's a truly awful game.",
       'Hi @EAHelp I’ve had Madeleine McCann in my cellar for the past 13 years and the little sneaky thing just escaped whilst I was loading up some fifa points, she took my card and I’m having to use my paypal account but it isn’t working, can you help me resolve it please?',
       'Thank you @EAMaddenNFL!! \n\nNew TE Austin Hooper in the ORANGE & BROWN!! \n\n#Browns | @AustinHooper18 \n\n pic.twitter.com/GRg4xzFKOn',
       'Rocket League, Sea of Thieves or Rainbow Six: Siege🤔? I love playing all three on stream but which is the best? #stream #twitch #RocketLeague #SeaOfThieves #RainbowSixSiege #follow',
       'my ass still knee-deep in Assassins Creed Odyssey with no way out anytime soon lmao',
       'FIX IT JESUS ! Please FIX IT ! What In the world is going on here.  @PlayStation @AskPlayStation @

### Text Preprocessing

In [62]:
processed_tweets=[]
for tweet in tweets:
    tweet=re.sub(r'[^a-zA-Z]',' ',tweet)
    tweet=tweet.lower()
    tweet=[word for word in tweet.split(' ') if not word in stopwords.words('english')]
    tweet=[stemmer.stem(word) for word in tweet]
    tweet=[word for word in tweet if len(word)!=0]
    tweet=' '.join(tweet)
    processed_tweets.append(tweet)

In [63]:
data=tfidf.fit_transform(processed_tweets)

# Creating a DataFrame to visualize the TF-IDF vectors with feature names
df_ = pd.DataFrame(data.toarray(), columns=tfidf.vocabulary_)
df_  # Displaying the resulting DataFrame

,microsoft,pay,word,function,poorli,samsungu,chromebook,csgo,matchmak,full,...,guess,areahg,overcharg,scam,mqb,bupthx,drink,wine,sun,fraction
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.193864,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
